In [10]:
import os # type: ignore
import time # type: ignore
import random # type: ignore
import sys # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore

from tqdm import tqdm # type: ignore

import torch # type: ignore
import torch.nn as nn # type: ignore
import torch.nn.functional as F # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import Dataset, DataLoader # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
import uproot as ur # type: ignore
import pickle # type: ignore

print("System Platform: ", sys.platform)
print('System Python Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Numpy version', np.__version__)

System Platform:  linux
System Python Version: 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
PyTorch version 2.6.0
Numpy version 2.1.3


Preliminary Data Manipulation

In [2]:
X, y = None, None
pickled_data_file_path = "transformed_data.pkl"
pickled_labels_file_path = "transformed_labels.pkl"
if os.path.exists(pickled_data_file_path) and os.path.exists(pickled_labels_file_path):
    with open(pickled_data_file_path, 'rb') as f:
        X = pickle.load(f)
    with open(pickled_labels_file_path, 'rb') as f:
        y = pickle.load(f)

In [ ]:
if X is None or y is None:
    background_data_file_name = "train_bkg_data_sideBands_lowQ_wPreselBDT_v5.root"
    signal_data_file_name = "train_sig_rare_lowQ_wPreselBDT_v6.root"

    background_data_file = ur.open(background_data_file_name)
    signal_data_file = ur.open(signal_data_file_name)
    features = ['Bprob', 'BsLxy', 'L2iso/L2pt', 'Bcos', 'Kiso/Kpt', 'LKdz', 'LKdr', 'Passymetry', 'Kip3d/Kip3dErr', 'L1id', 'L2id']
    sample_weights = 'trig_wgt'
    preselection = '(KLmassD0 > 2.) & ((Mll>1.05) & (Mll<2.45))'

    sig_dict = signal_data_file['mytree'].arrays(features, library='np', cut=preselection)
    bkg_dict = background_data_file['mytree'].arrays(features, library='np', cut=preselection)
    backgr = np.stack(list(bkg_dict.values()))
    signal = np.stack(list(sig_dict.values()))

    X = np.transpose(np.concatenate((signal, backgr), axis=1))
    y = np.concatenate((np.ones(signal.shape[1]), np.zeros(backgr.shape[1])))

    with open(pickled_data_file_path, 'wb') as f:
        pickle.dump(X, f)
    with open(pickled_labels_file_path, 'wb') as f:
        pickle.dump(y, f)

In [12]:
random_seed = 7

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=random_seed)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=random_seed)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

Training set: (2553918, 11)
Validation set: (547268, 11)
Test set: (547269, 11)


Dataset & DataLoader Instantiation

In [5]:
class Transform:
    def __call__(self, data: np.ndarray):
        return data

class Normalize(Transform):
    def __call__(self, data: np.ndarray):
        data = data.astype(np.float64)

        min_values = np.min(data, axis=0)
        max_values = np.max(data, axis=0)

        data -= min_values
        data /= (max_values - min_values)

        return data


In [13]:
class ParticlesDataset(Dataset):
    def __init__(self, data: np.ndarray, labels: np.ndarray, transform: Transform = Transform()):
        if data.shape[0] != labels.shape[0]:
            raise RuntimeError("Training data and training labels have size mismatch:", self.__class__.__name__)
        
        self.data = torch.tensor(transform(data), dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return (self.data[index], self.labels[index])
    
    @property
    def classes(self):
        return ['Background', 'Signal']
    
    @property
    def features(self):
        return ['Bprob', 'BsLxy', 'L2iso/L2pt', 'Bcos', 'Kiso/Kpt', 'LKdz', 'LKdr', 'Passymetry', 'Kip3d/Kip3dErr', 'L1id', 'L2id']
    

In [14]:
training_dataset = ParticlesDataset(data=X_train, labels=y_train, transform=Normalize())
training_dataloader = DataLoader(dataset=training_dataset, batch_size=64, shuffle=True)

validation_dataset = ParticlesDataset(data=X_val, labels=y_val, transform=Normalize())
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=64, shuffle=False)

test_dataset = ParticlesDataset(data=X_test, labels=y_test, transform=Normalize())
test_dataloader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

Shallow/Narrow Architecture:
This will be a very simple and smaller model that will act as a performance baseline. It won't have very many layers or very many nodes in each layer. 
Below is the original starting point I used for this architecture, this may be subject to change. 

Original Layers:

    Input ReLU Linear 12 -> 32

    Hidden ReLU Linear 32 -> 16

    Output Sigmoid Linear 16 -> 1

In [ ]:
class ShallowNarrowModel(nn.Module):
    def __init__(self):
        super(ShallowNarrowModel, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(11,32),
            nn.ReLU(),
            nn.Linear(32,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
shallow_narrow_model = ShallowNarrowModel()
num_epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=shallow_narrow_model.parameters(), lr=0.001)
training_losses, validation_losses = [], []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
shallow_narrow_model.to(device)

In [ ]:
for epoch in num_epochs:
    shallow_narrow_model.train()
    running_loss = 0.0

    for events, labels in tqdm(training_dataloader, desc="Training Loop: Shallow Narrow Model"):
        events, labels = events.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = shallow_narrow_model(events)
        loss = criterion(outputs, labels)
        loss.backwards()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)

    train_loss = running_loss / len(training_dataloader.dataset)
    training_losses.append(train_loss)

    shallow_narrow_model.eval()
    running_loss = 0.0

    with torch.no_grad():
        for events, labels in tqdm(validation_dataloader, desc="Validation Loop: Shallow Narrow Model"):
            events, labels = events.to(device), labels.to(device)
            outputs = shallow_narrow_model(events)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)

        val_loss = running_loss / len(validation_dataloader.dataset)
        validation_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")

In [ ]:
plt.plot(training_losses, label='Training loss')
plt.plot(validation_losses, label='Validation loss')
plt.legend()
plt.title("Loss over epochs")
plt.show()

Deep/Narrow Architecture:
There will be more layers in this model, but the amount of nodes per layer will remain relatively smaller. The amount of nodes per layer will also remain balanced.
Below is the original starting point I used for this architecture, this may be subject to change. 

Original Layers:

    Input ReLU Linear 12 -> 32

    Hidden ReLu Linear 32 -> 32

    Hidden ReLu Linear 32 -> 32

    Hidden ReLu Linear 32 -> 32

    Hidden ReLu Linear 32 -> 32

    Hidden ReLu Linear 32 -> 32

    Hidden ReLu Linear 32 -> 32

    Hidden ReLU Linear 32 -> 16

    Output Sigmoid Linear 16 -> 1

Shallow/Wide Architecture: There will be relatively less layers in this one, but more nodes in each one. 
Below is the original starting point I used for this architecture, this may be subject to change. 

Original Layers:

    Input ReLU Linear 12 -> 64

    Hidden ReLU Linear 64 -> 128

    Hidden ReLU Linear 128 -> 64

    Hidden ReLU Linear 64 -> 32

    Hidden ReLU Linear 32 -> 16

    Output Sigmoid Linear 16 -> 1